In [2]:
"""
Note about project:  This project is exploratory. I plan to run regression on students' performance and their 
attitude towards education. Other factors aside from attitude, such as socioeconomic status, definitely affect 
education performance regardless of students' attitude. 

Goal of this project: Test which questions strongly correlate with performance to help with the future data collection 
by TDRI.  If we have a general idea on which specific attitudes correlate with education, we can design a survey 
that better captures the result of education interventions. 
"""
# Importing all the necessary libraries. 
import pandas as pd
import numpy as np
import statsmodels.api as sm


In [3]:
"""
Input: List of numbers
Output: The average of members within the list.
Description: This function helps when I apply conditions to the dataframe, such as separating students into those
that attend public and private schools. In some instances, the length of the dataframe is 0, and dividing 0 gives
an error. Because I prefer to abstract from checking the type of elements within each array, I create this function. 
""" 

def avg(lst):
    count = 0
    sum_val = 0
    for member in lst:
        try:
            temp = float(member)
            count += 1
            sum_val += temp
        except:
            temp = 0
    try:
        return sum_val/count
    except:
        return 0

In [4]:
# Reading stata file. Note that columns with all N/A cell will be dropped from this dataset. 
# I have experimented with dropping columns with any N/A cell, but the final result was too many dataset is dropped.
"""
Notes on data:
1. This data is a survey administered to approximately 8600 students attending 290 different schools. Some of
the columns contain categorical variables, and others ordinal/numerical variables. This suggests the need to carefully
select which datafield will be included in the final analysis.
2. Some of the cells contain No Response/Not available. I must decide later on how to impute/remove these data
before running the regression model. 
"""

thadf = pd.read_stata("PISA_2018_THA.dta").dropna(how = "all", axis = 1).fillna(method = "ffill").fillna(method = "bfill")

In [5]:
# Creatinga a dictionary whose key is column name and value is column interpretation. 
# This method helps because when I tried reading .dta files directly, some of the values change. 
"""
Credit: https://stackoverflow.com/questions/44809696/is-there-a-way-to-read-stata-labels-in-python
"""

reader = pd.io.stata.StataReader("PISA_2018_THA.dta")
header = reader.variable_labels()
for var in header:
        name  = var
        label = header[name]

In [6]:
# Eliminating the keys of columns we have already dropped so that we can take advantage of .columns.tolist() library.

header_temp = {}
for key in list(thadf.columns.tolist()):
    header_temp[key] = header[key]

header = header_temp 

In [7]:
# Creating a flipped dictionary so that we can search for the column name if we have an aspect of data we are 
# interested in. 

header_flip = {}
for key in list(header.keys()):
    header_flip[header[key]] = key

In [8]:
# Creating three new columns -- math, read, scie, glcm -- to calculate the average statistics for each student.  
"""
Note: Global competency (GLCM) is dropped because there is no clear way to calculate the competency. For instance,
some questions have a lot of No Responses; others including questions that are not taught in Thai curriculum. 
Therefore, I focused on mathematics, science, and reading because these subjects are less context-dependent.
"""

num_row = thadf.shape[0]
thadf = thadf.assign(math = [0]*num_row, read = [0]*num_row, scie = [0]*num_row) 
col_lst = thadf.columns.tolist()

In [9]:
# Calculating the summary statistics for mathematics and science by averaging scores for all 10 levels in the subject. 

subject_lst = ["math", "scie"]
temp = "pv"
for index in thadf.index.tolist():
    for subject in subject_lst:
        sum_score = 0
        for i in range(1,11):
            col_name = temp + str(i) + subject
            sum_score += thadf.loc[index, col_name]
        avg_score = sum_score/10
        thadf.loc[index, subject] = avg_score

In [10]:
# Calculating the summary statistics for reading by averaging scores for all 10 levels across different areas of
# reading. 
"""
Each subfield of reading is as follows:
1.) rtsn = Single Text Structure. 
2.) rcer = Evaluate and Reflect 
3.) Multiple Text Structure
4.) rcun = Reading comprehenseion 
5.) rcli = Locate information  

Note: This analysis could be more detailed. For instance, we could treat reading comprehension and locating 
information as separate fields of data. However, I chose to average their assessment score to understand a larger
trend of Thai students' reading skills. 
"""
read_lst = ["rtsn", "rcer","rtml", "rcun", "rcli"]
for index in thadf.index.tolist():
    sum_score = 0
    for read in read_lst:
        for i in range(1,11):
            col_name = temp + str(i) + read
            sum_score += thadf.loc[index, col_name]
    avg_score = sum_score/50 # Average over 50 types of reading
    thadf.loc[index, "read"] = avg_score
subject_lst = subject_lst + ["read"]

In [11]:
"""
At this point, I will run a regression project on students' attitude.

Note: Expected maximum education attainment has some idiosyncrasy. There are 6000+ students who check bachelor's
degree, but do not check middle school and high school. Therefore, I chose to exclude this data from the analysis.
"""

# Step 1: Create a list of questions pertaining to attitude by grouping relevant questions altogether into 
# different groups. 

content = ["st016q01na"]
school_try = ["st036q05ta","st036q06ta","st036q08ta"]
max_edu = ["st225q01ha","st225q02ha","st225q03ha", "st225q04ha","st225q06ha"]
compete = ["st181q02ha", "st181q03ha", "st181q04ha"]
failure = ["st183q01ha","st183q02ha","st183q03ha"]
growth = ["st184q01ha"]
life_meaning = ["st185q01ha","st185q02ha","st185q03ha"]
mental_health = []
for i in range(1,10):
    temp = "st186q0"
    if (i != 4):
        temp += str(i) + "ha"
        mental_health.append(temp)
mental_health.append("st186q10ha")
mastery = ["st208q01ha","st208q02ha","st208q04ha"]
self_manage = []
for i in range(1,8):
    temp = "st188q0"
    if (i != 5 and i != 4):
        temp += str(i) + "ha"
        self_manage.append(temp)
school_exp = []
for i in range(1,7):
    temp = "st034q0"
    if (True):
        temp += str(i) + "ta"
        school_exp.append(temp)     

In [12]:
# Merging each list of question groups altogether. 
attdf_questions = content + school_try + max_edu + compete + failure + growth + life_meaning + mental_health + mastery
attdf_questions += self_manage + school_exp

In [13]:
# Step 2: Create a dictionary where we check how many misses/no responses are there in each variable.
"""
Note: no_resp_Qkey is a dictionary whose key is the question_id and value is the number of total 'No Response' from 
every student. no_resp_Skey is a dictionary whose key is the student_id and value the number of 'No Response' that
student provides in total. 
"""

attdf = thadf[attdf_questions]

no_resp_Qkey = {} 
no_resp_Skey = {}

for question in attdf_questions:
    no_respdf = attdf[attdf[question] == "No Response"]
    no_resp_Qkey[question] = no_respdf.shape[0]

for index in attdf.index.tolist(): 
    no_respdf = attdf.loc[index]
    for question in attdf_questions:
        try:
            no_resp_Skey[index] += 1
        except:
            no_resp_Skey[index] = 1

In [34]:
"""
After checking the no_resp_Qkey, I decided not to drop any questions because questions which have the highest
'No Response' perform pretty well with approximate 250 misses out of the total 8600 responses. Therefore, I will
drop students who provide a lot of 'No Response' instead, which in this case, I chose to drop any student with
more than 1 'No Response'.
"""

count = 0
temp = 2
for mem in list(no_resp_Skey.values()):
    if mem >= temp:
        count += 1

print(count)

8633


In [73]:
# Step 3: Create a new dictionary for each student whereby agree/disagree is converted to 1-4 scale where
# 1 = Strongly disagree/Never; 2 = Disagree/Rarely; 3 = Agree/Sometimes; 4 = Strongly agree/Always.

"""
Note: Because responses to each question is ordinal (1-4/1-5 scale on disagree-strongly agree), I created a new dataframe 
to run linear regression. I specifically used the to_dict method to take advantage of the fast computation
of the to_list method on each column. To deal with 'No Response', I chose to impute the cell with the responses 
that appear the most commonly among the responses given. Because there are not many 'No Response' in the first place,
this method would not significantly affect the final result of the regression model. 
"""

data = {}
# I created an expected_edu_lst to exclude these responses from the regression model. The reason is that some of the 
expected_edu_lst = ["st225q01ha","st225q02ha","st225q03ha","st225q04ha","st225q06ha"]

for question in attdf_questions: 
    # This question 'st016q01na' is a numerical variable., so we can directly copy the value into a new array 
    # or impute. 
    if question == 'st016q01na':
        temp_lst = []
        for response in attdf[question].tolist():
            # Directly adding the value to the list if the student provides some response.
            if response != "No Response":
                temp_lst.append(response)
            else:
            # Else, impute the No Response with the most common answers that appeared in the column.  
                most_common = attdf[question].value_counts().index.tolist()[0]
                temp_lst.append(most_common)
    # If the question is ordinal variable, we need to convert into 1-4/1-5 scale first. 
    elif question not in subject_lst and question not in expected_edu_lst:
        temp_lst = []
        for response in attdf[question].tolist():
            if response == "No Response":
                #  Impute the No Response with the most common answers that appeared in the column.  
                response = attdf[question].value_counts().index.tolist()[0]
            if response in ["Strongly disagree","Never","Not at all true of me"]:
                temp_lst.append(1)
            elif response in ["Disagree", "Rarely", "Slightly true of me"]:
                temp_lst.append(2)
            elif response in ["Agree", "Sometimes", "Moderately true of me"]:
                temp_lst.append(3)
            elif response in ["Strongly agree", "Always","Very true of me"]:
                temp_lst.append(4) 
            elif response == "Extremely true of me":
                temp_lst.append(5)
    data[question] = temp_lst

for subject in subject_lst:
    data[subject] = thadf[subject].tolist()

In [74]:
# Create a new dataframe where every attitude-related question is converted into numbers. 
replace_attdf = pd.DataFrame.from_dict(data)

In [75]:
replace_attdf

,st016q01na,st036q05ta,st036q06ta,st036q08ta,st225q01ha,st225q02ha,st225q03ha,st225q04ha,st225q06ha,st181q02ha,...,st188q07ha,st034q01ta,st034q02ta,st034q03ta,st034q04ta,st034q05ta,st034q06ta,math,scie,read
0,7.0,3,3,3,3,3,3,3,3,3,...,3,2,3,2,3,3,3,609.2849,597.3111,598.41500
1,7.0,3,3,3,3,3,3,3,3,3,...,2,2,3,3,2,3,2,641.9133,580.9458,563.66300
2,10.0,4,4,4,4,4,4,4,4,3,...,3,1,3,4,1,3,1,573.5735,502.3006,516.70010
3,3.0,3,4,3,3,3,3,3,3,2,...,3,3,3,3,2,2,2,517.8524,521.9634,460.82614
4,3.0,4,4,4,4,4,4,4,4,2,...,4,1,4,3,3,3,2,606.0296,515.3353,500.12668
5,8.0,1,1,1,1,1,1,1,1,3,...,3,2,3,4,2,3,2,576.1993,537.6613,550.94154
6,3.0,3,4,4,4,4,4,4,4,3,...,2,3,2,2,3,2,4,546.0253,573.0372,529.91428
7,10.0,4,4,4,4,4,4,4,4,3,...,4,1,3,3,1,3,1,554.5563,509.9338,478.41574
8,7.0,4,4,4,4,4,4,4,4,3,...,3,1,4,3,1,3,1,487.9661,499.6684,477.81310
9,7.0,2,2,3,3,3,3,3,3,2,...,3,3,4,2,2,3,2,589.4993,576.4603,598.00048


In [78]:
# Step 4: Replace ordinal variables on disagree-agree. 
"""
Note: The constant is added because every student should have a based intelligent even before entering school. 
"""

X = replace_attdf[attdf_questions]
y = replace_attdf["math"]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
predictions = model.predict(X) 

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   math   R-squared:                       0.317
Model:                            OLS   Adj. R-squared:                  0.314
Method:                 Least Squares   F-statistic:                     107.7
Date:                Fri, 24 Jan 2020   Prob (F-statistic):               0.00
Time:                        01:57:07   Log-Likelihood:                -49840.
No. Observations:                8633   AIC:                         9.976e+04
Df Residuals:                    8595   BIC:                         1.000e+05
Df Model:                          37                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        424.4081     11.517     36.849      0.000     401.831     446.985
st016q01na    -3.4475      0.430     -8.020      0.000      -4.290      -2.605
st036q05ta   -10.3293      1.939     -5.327      0.000     -14.131      -6.528
st036q06ta    11.2452      1.943      5.789      0.000       7.437      15.053
st036q08ta     0.4099      0.321      1.277      0.202      -0.219       1.039
st225q01ha     0.4099      0.321      1.277      0.202      -0.219       1.039
st225q02ha     0.4099      0.321      1.277      0.202      -0.219       1.039
st225q03ha     0.4099      0.321      1.277      0.202      -0.219       1.039
st225q04ha     0.4099      0.321      1.277      0.202      -0.219       1.039
st225q06ha     0.4099      0.321      1.277      0.202      -0.219       1.039
st181q02ha   -10.1423      1.314     -7.720      0.000     -12.718      -7.567
st181q03ha    -9.1295      1.415     -6.451      0.000     -11.904      -6.355
st181q04ha    19.5894      1.493     13.125      0.000      16.664      22.515
st183q01ha    10.5343      1.527      6.899      0.000       7.541      13.528
st183q02ha    10.3375      1.616      6.396      0.000       7.169      13.506
st183q03ha   -10.8244      1.365     -7.929      0.000     -13.500      -8.148
st184q01ha   -25.0090      0.973    -25.704      0.000     -26.916     -23.102
st185q01ha    -9.5136      1.799     -5.288      0.000     -13.040      -5.987
st185q02ha    -8.0663      1.925     -4.191      0.000     -11.839      -4.293
st185q03ha    -2.8066      1.889     -1.486      0.137      -6.510       0.897
st186q01ha    -3.2989      1.796     -1.837      0.066      -6.819       0.221
st186q02ha    19.9612      1.534     13.011      0.000      16.954      22.969
st186q03ha    -2.4772      1.882     -1.316      0.188      -6.167       1.212
st186q05ha    19.8731      1.865     10.655      0.000      16.217      23.529
st186q06ha    -7.4950      1.348     -5.559      0.000     -10.138      -4.852
st186q07ha     7.1185      1.814      3.924      0.000       3.563      10.674
st186q08ha   -12.0733      1.583     -7.629      0.000     -15.175      -8.971
st186q09ha   -11.0164      1.517     -7.261      0.000     -13.991      -8.042
st186q10ha    -1.7001      1.609     -1.057      0.291      -4.854       1.453
st208q01ha    -2.9013      1.379     -2.105      0.035      -5.603      -0.199
st208q02ha     4.7144      1.577      2.989      0.003       1.623       7.806
st208q04ha     6.1970      1.554      3.988      0.000       3.151       9.243
st188q01ha     9.0546      1.992      4.545      0.000       5.150      12.959
st188q02ha    23.3626      1.829     12.774      0.000      19.777      26.948
st188q03ha    -3.7424      1.439     -2.600      0.009      -6.564      -0.921
st188q06ha     3.9051      1.892      2.064      0.039       0.197       7.613
st

In [48]:
# Step 5: Summarize results
# Summary of methodology and result. 
"""
Methodology: 
First, I loaded the PISA 2018 data on Thai students' education achievement. Because people have previously studied 
the impact of students' socioeconomic background on their education achievement, I decided to focus on questions
pertaining to students' attitude towards education (referring to P.47-P.60 of the student questionnaire). 

Second, I looked at the data and observed that virtually every question is on an Agree/Disagree scale except for 
their expected education achievement. After looking at the expected education level, I concluded that the data 
was not properly filled since it added up to more than 8600 (the total number of students participating in this
study). Therefore, I decided to remove it for the regression analysis. Moreover, we experienced several problems
with No Response, so we filter out every student whose responses miss at least 2 spots. Not much data is missed 
given that only 227 out of 8700 students have at least 2 spots missed; for others, their data is complete or 
misses only one spot. 

Third, I converted the Agree/Disagree questionnaire into 1-4 scale and ran multiple linear regression. The result
will be shown above. Only the variables with p-value less than 0.05 will be shown. 

Fourth, group the variables by positive/negative coefficient and pick only the strongest coefficients. 
That is, pick variables that have high explanatory power.

Limitation: Reverse causality (e.g. Dumbness may cause fixed mindset. )

Idea: Check the interaction between socioeconomic background and A) Economic B) Growth Mindset. Will do so soon. 
"""

positive_lst = ["st036q06ta","st181q04ha","st183q01ha","st183q02ha","st186q02ha","st186q05ha","st186q07ha"]
positive_lst += ["st188q01ha","st188q02ha"]
negative_lst = ["st036q05ta","st181q02ha","st181q03ha","st183q03ha","st184q01ha","st185q01ha","st185q02ha"]
negative_lst += ["st186q06ha","st186q08ha","st186q09ha","st188q07ha","st034q04ta"]
print("----- Positive lst ----- ")
for member in positive_lst:
    print(header[member])
print("----- Negative lst ----- ")
for member in negative_lst:
    print(header[member])

# Interpretation:
# Theme 1: External motivation. Students are motivated by values in more immediate future in which they have more 
# control (e.g. good university >> good job). Other people's perception
# Theme 2: Healthy concern correlate with high education performance. Factors such as life contentment, satisfaction,
# and life meaning negatively correlate with education performance. 
# Theme 3: Emotion state. Concerned and happy/joyous/content students are top-performers. 
# Theme 4: Growth mindset. Predicts strong performance. 
# Overall: Healthy concern (i.e. not too complacent) + growth mindset + external motivation + happy emotion state. 

----- Positive lst ----- 
Thinking about your school: Trying hard at school will help me get into a good <
Agree: I try harder when I’m in competition with other people.
Agree: When I am failing, I worry about what others think of me.
Agree: When I am failing, I am afraid that I might not have enough talent.
Thinking about yourself and how you normally feel: how often do you feel as desc
Thinking about yourself and how you normally feel: how often do you feel as desc
Thinking about yourself and how you normally feel: how often do you feel as desc
Agree: I usually manage one way or another.
Agree: I feel proud that I have accomplished things.
----- Negative lst ----- 
Thinking about your school: Trying hard at school will help me get a good job.
Agree: I enjoy working in situations involving competition with others.
Agree: It is important for me to perform better than other people on a task.
Agree: When I am failing, this makes me doubt my plans for the future.
Agree: Your intelligence 